<a href="https://colab.research.google.com/github/shubhamx64/SBHacks2019/blob/master/Jasmine8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from google.colab import drive
import pickle
from nltk.tokenize import sent_tokenize, word_tokenize

drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
options = { 'node_color': 'black','node_size': 100,'width': 3}

In [0]:
#Translate sentence to a Scene
debug = 1
class Scene():
  #Each sentece is a temporal evolution of graphs
  def __init__(self):
    self.node_list = []
    self.relationList = []
    self.sceneGraph = nx.DiGraph()
    self.temporalListofGraphs = []
    self.timeAlive = 0 #No concept of time (yet)
  
  def addNode(self, node):
    if debug == 1: 
      print("Debug: added node: ", node)
    self.node_list.append(node)
    self.sceneGraph.add_node(node)
    
  def addEdgeAttributeName(self, attr, f, t):
    if debug == 1:
      print("Debug: Added edge between", f, " and ", t, " with  attr: ", attr)
    self.sceneGraph.add_edge(f,t, aName=attr, tAdded=self.timeAlive)
    self.relationList.append([f,t,attr])
    
  def drawGraphatTime(self, time):
    print(self.temporalListofGraphs)
    pos = nx.shell_layout(self.temporalListofGraphs[time]);
    nx.draw_random(self.temporalListofGraphs[time], with_labels=True)
    
  def drawCurrentGraph(self):
    pos = nx.shell_layout(self.temporalListofGraphs[self.timeAlive-1])
    nx.draw_random(self.temporalListofGraphs[self.timeAlive-1], with_labels=True)
  
  def getOwnerOfThing(self, thing):
    predSet = self.sceneGraph.predecessors(thing)
    setOfNodes = []
    for n in predSet:
      setOfNodes.append([n, self.sceneGraph[n][thing]['tAdded']])
    maxTime = 0
    maxIndex = 0
    for x in range(len(setOfNodes)):
      if int(setOfNodes[x][1]) > maxTime:
        maxTime = setOfNodes[x][1]
        maxIndex = x
    return setOfNodes[maxIndex][0]
    
  def captureMoment(self):
    #NOTE: Only incriment time here and no where else 
    self.temporalListofGraphs.append(self.sceneGraph.copy())
    self.timeAlive = self.timeAlive + 1
    
  def answerQuestion(self, fromE, attrib):
    succSet = self.sceneGraph.successors(fromE)
    setOfNodes = []
    for n in succSet:
      if self.sceneGraph[fromE][n]['aName'] == attrib:
        setOfNodes.append([n, self.sceneGraph[fromE][n]['tAdded']])
    maxTime = 0
    maxIndex = 0
    for x in range(len(setOfNodes)):
      if int(setOfNodes[x][1]) > maxTime:
        maxTime = setOfNodes[x][1]
        maxIndex = x
    return setOfNodes[maxIndex][0]
      
        
    
    
    

In [0]:
class Concept():
  #Each concept has a name and a series of ActionMemories
  def __init__(self, cName, a, f, t, attrib, tag=None):
    self.conceptName = cName
    self.word_list = []
    self.action = a
    self.connectThis = f
    self.connectTo = t
    self.attribName = attrib
    self.lexCategory = tag # the lexical category of the word eg. noun,verb..
    
  def addWordToConcept(self, w):
    self.word_list.append(w)


In [0]:
class Question():
  def __init__(self, q, atribAns):
    self.questionTag = q
    self.attribAsAnswer = atribAns

In [0]:
class Mind():
  #Mind is the object that is saved and restored so contain state-less
  #information here
  def __init__(self):
    self.listOfConcepts = []
    self.listOfQues = []
  
  def addConcept(self, c):
    self.listOfConcepts.append(c)
    
  def addQuestion(self, q):
    self.listOfQues.append(q)

class Jasmine():
  def __init__(self):
    #Recover memory
    self.mind = Mind()
    self.retriveMemories()
    self.mainScene = Scene()
    
  def resetMemory(self):
    self.mind = Mind()
    
  def saveMemories(self):
    with open('/content/gdrive/My Drive/memoryState.obj', 'wb') as file:
      pickle.dump(self.mind, file)
      
  def retriveMemories(self):
    with open('/content/gdrive/My Drive/memoryState.obj', 'rb') as file:
      self.mind = pickle.load(file)
  
  def parseQuestion(self, ques):
    words = [word for word in word_tokenize(ques)]
    found = 0
    #Looking for answer attribute
    ansAtr = ""
    qWord = words[0]
    for ques in self.mind.listOfQues:
      if ques.questionTag == qWord and found == 0:
        ansAtr = ques.attribAsAnswer
        found = 1
    if found == 0:
      self.sayThis("I don't know what " + qWord + " means")
      ans = self.askThis("Can you teach me? ")
      if ans == "Yes" or ans == "yes":
        aAtr = self.askThis("What is the answer to it generally? ")
        self.mind.addQuestion(Question(qWord, aAtr))
        
    #Look for which object to find attrib of
    for word in words:
      x = self.word2Concept(word) 
      if x is not None:
        if x.conceptName == "Person":
          subject = word
        if x.conceptName == "Thing":
          #subject = #select latest owner and move forward
          subject = self.mainScene.getOwnerOfThing(word)
        
    #Search where the edge with the name above leads to
    ans = self.mainScene.answerQuestion(subject, ansAtr)
    self.sayThis(ans)

  
  def readUserInput(self):
    currentInput = input("You: ")
    #Check if question or scene (./?)
    if currentInput.endswith('.'):
      self.parseSentence2Scene(currentInput)
    elif currentInput.endswith('?'):
      #Parse question to string 
      self.parseQuestion(currentInput)
  
  def sayThis(self, str):
    print("Jasmine: ", str)
    
  def askThis(self, str):
    self.sayThis(str)
    return input("You: ")
    
  def executeThis(self, concept, instanceName):
    if concept.action == "add node":
      self.mainScene.addNode(instanceName)
    elif concept.action == "add edge":
      self.mainScene.addEdgeAttributeName(concept.attribName, concept.connectThis, concept.connectTo)
    self.mainScene.captureMoment()
    
  def word2Concept(self, w):
    for concept in self.mind.listOfConcepts:
      for word in concept.word_list:
        if word == w:
          return concept
    return None
    
  def createSpecficConceptInstance(self, con, wordList):
    #Search the concept stored in connectThis in
    lookingFor = con.connectThis
    for word in wordList:
      x = self.word2Concept(word) 
      if x is not None:
        if x.conceptName == lookingFor:
          conF = word
    lookingFor = con.connectTo
    for word in wordList:
      x = self.word2Concept(word) 
      if x is not None:
        if x.conceptName == lookingFor:
          conT = word
    return Concept(con.conceptName, con.action, conF, conT, con.attribName)
        
          
    
  def parseSentence2Scene(self,currentInput):
    posTag = ["NNP", # proper noun , eg:Mike
              "NN", #noun , eg:desk
              "VB", #verb(base form) eg:move
              "VBD"] #verb past tense
    found = 0
    words = [word for word in word_tokenize(currentInput)]
    tags = nltk.pos_tag(words)
    print(tags)
    wordCnt = 0
    for word in words:
      #Search memory using your mind 
      #if found then execute otherwise learn with human
      #if an edge already exists in the scene:   
      found = 0
      tag = tags[wordCnt][1]
      for i in range(len(posTag)):
        if tag == posTag[i] :
          print(i , tag)
          indexTag =i
      wordCnt = wordCnt +1
      
      entities = nltk.chunk.ne_chunk(tags)
      print(entities)
      for concept in self.mind.listOfConcepts:
        for wordSeen in concept.word_list:
          if wordSeen == word:
            #Word found in action memory 
            #execute action in the mind
            found = 1
            if(concept.action != "add edge"):
              self.executeThis(concept, word)
      if(found == 0):
        #Word not found ask human L0 question 
        self.sayThis("I don't know what " + word + " means.")
        ans = self.askThis("Can you teach me?")
        #Add to static memory systematically 
        if ans == "Yes" or ans == "yes":
          self.learnConceptWithHuman(word, words)
          
    #Now you have plotted nodes, plot edges
    for word in words:
      for concept in self.mind.listOfConcepts:
        for wordSeen in concept.word_list:
          if wordSeen == word:
            #Word found in action memory 
            #execute action in the mind
            found = 1
            if(concept.action == "add edge"):
              newConcept = self.createSpecficConceptInstance(concept, words)
              self.executeThis(newConcept, word)
  
  def learnConceptWithHuman(self, word, words):
    ans = self.askThis("Do I Know the Concept?")
    if ans == "Yes" or ans == "yes":
      #Search in concept memory and append word instance
      cName = self.askThis("What is the name of the concept?")
      for concept in self.mind.listOfConcepts:
        if concept.conceptName == cName:
          concept.addWordToConcept(word)
    else:
      #create a new concept 
      cName = self.askThis("What is the name of the concept?")
      act = self.askThis("What is the action I am supposed to take?")
      fName = None
      tName = None
      aName = None
      if act == "add edge":
        fName = self.askThis("What does it link?")
        tName = self.askThis("What does it link to?")
        aName = self.askThis("What is the connection?")
      con = Concept(cName, act, fName, tName, aName)
      con.addWordToConcept(word)
      self.mind.addConcept(con)
    
  def makeSelfAware(self):
    self.mainScene.add_node("Jasmine")
    self.mainScene.captureMoment()
    
    
    
    

1 Mary moved to the bathroom.
2 John went to the hallway.
3 Where is Mary? 	bathroom	1
4 Daniel went back to the hallway.
5 Sandra moved to the garden.
6 Where is Daniel? 	hallway

In [0]:
j = Jasmine()
#j.resetMemory()


In [29]:
for _ in range(6):
  j.readUserInput()
j.saveMemories()

KeyboardInterrupt: ignored

In [0]:
j.mainScene.drawCurrentGraph()